In [ ]:
!python3 --version

In [ ]:
import os
from pathlib import Path

In [ ]:
import pandas as pd
import numpy as np
import cv2 as cv

In [ ]:
# Dataset Paths
DATASET_PATH = Path('/mnt/DATA/fharookshaik/major_project/dataset')
TRAIN_IMAGES_DIR_PATH = os.path.join(DATASET_PATH,'Train','images')

TRAIN_CSV_PATH = os.path.join(DATASET_PATH,'Train','train.csv')
VALIDATE_CSV_PATH = os.path.join(DATASET_PATH,'Train','val.csv')

TEST_IMAGES_DIR_PATH = os.path.join(DATASET_PATH,'Test','images')
TEST_CSV_PATH = os.path.join(DATASET_PATH,'Test','test.csv')

In [ ]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
train_df.head()

In [ ]:
img = train_df['image'][0]
img = os.path.join(TRAIN_IMAGES_DIR_PATH,img)
img

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

In [ ]:
base_model = tf.keras.applications.InceptionV3(
    include_top=True,
    weights = "imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classifier_activation="softmax"
)

In [ ]:
im = image.load_img(img,target_size=(299,299))
x = image.img_to_array(im)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

features = base_model.predict(x)

In [ ]:
decode_predictions(features, top=3)

In [ ]:
# x = base_model.output
# x = GlobalAveragePooling2D()(x)

# x = Dense(1024,activation='relu')(x)

# predictions = Dense(200,activation='softmax')(x)

# model = Model(inputs=base_model.input,outputs=predictions)

# for layer in base_model.layers:
#     layer.trainable = False

# model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

In [1]:
import os
import pandas as pd
from pathlib import Path

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fharookshaik\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
stemmer = PorterStemmer()
words = stopwords.words("english")

In [3]:
# Dataset Paths
DATASET_PATH = Path(r'D:\fharookshaik\major_project\dataset')
TRAIN_IMAGES_DIR_PATH = os.path.join(DATASET_PATH,'Train','images')

TRAIN_CSV_PATH = os.path.join(DATASET_PATH,'Train','train.csv')
VALIDATE_CSV_PATH = os.path.join(DATASET_PATH,'Train','val.csv')

TEST_IMAGES_DIR_PATH = os.path.join(DATASET_PATH,'Test','images')
TEST_CSV_PATH = os.path.join(DATASET_PATH,'Test','test.csv')

In [4]:
train_df = pd.read_csv(TRAIN_CSV_PATH)

In [5]:
train_df.head()

,OCR,image,hero,villain,victim,other
0,Bernie or Elizabeth? Be informed. Compare them...,covid_memes_18.png,NaN,NaN,NaN,"['bernie sanders', 'elizabeth warren']"
1,Extending the Brexit deadline until October 31...,covid_memes_19.png,NaN,['uk government'],NaN,NaN
2,kwai gkwa 0964 #nnevvy applause to Thais from ...,covid_memes_252.png,['thais'],NaN,NaN,['hong kong']
3,"So, I order this face mask to protect against ...",covid_memes_255.png,NaN,['china'],NaN,"['face mask', 'made in china', 'coronavirus']"
4,best candidate for JA 2020 joe biden Kamala ha...,covid_memes_20.png,['joe biden'],NaN,NaN,"['bernie sanders', 'kamala harris', 'tiktok']"


In [6]:
train_df.dtypes

OCR        object
image      object
hero       object
villain    object
victim     object
other      object
dtype: object

In [8]:
import re

In [11]:
train_df['OCR'] = train_df['OCR'].astype(str)

In [12]:
train_df['procesed_OCR'] = train_df['OCR'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [13]:
train_df.head()

,OCR,image,hero,villain,victim,other,procesed_OCR
0,Bernie or Elizabeth? Be informed. Compare them...,covid_memes_18.png,NaN,NaN,NaN,"['bernie sanders', 'elizabeth warren']",berni elizabeth be inform compar issu matter i...
1,Extending the Brexit deadline until October 31...,covid_memes_19.png,NaN,['uk government'],NaN,NaN,extend brexit deadlin octob st order ensur dea...
2,kwai gkwa 0964 #nnevvy applause to Thais from ...,covid_memes_252.png,['thais'],NaN,NaN,['hong kong'],kwai gkwa nnevvi applaus thai hong kong whi th...
3,"So, I order this face mask to protect against ...",covid_memes_255.png,NaN,['china'],NaN,"['face mask', 'made in china', 'coronavirus']",so i order face mask protect corona viru hoool...
4,best candidate for JA 2020 joe biden Kamala ha...,covid_memes_20.png,['joe biden'],NaN,NaN,"['bernie sanders', 'kamala harris', 'tiktok']",best candid ja joe biden kamala harri berni sa...


In [28]:
OCR = train_df['OCR'].tolist()

5552

In [15]:
entities = []
actors = ['hero','villain','victim','other']
for idx,val in train_df.iterrows():
    # entities = []
    # print('image : ', val.get('image'))
    for actor in actors:
        temp_str = str(val.get(actor))
        if temp_str != 'nan':
            enty_list = temp_str.strip("][").split(', ')
            for enty in enty_list:
                entities.append(enty.strip("\'"))
    # print(entities)
    # print('-'*20)

In [16]:
OCR_enty = OCR + entities

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
vect = CountVectorizer()
train_tfIdf = vect.fit_transform(OCR)

x = dict(sorted(vect.vocabulary_.items(), key = lambda x: x[1], reverse = False))
print(x)

{'00': 0, '000': 1, '0000': 2, '00000': 3, '000000': 4, '002': 5, '004': 6, '00800': 7, '00am': 8, '00pm': 9, '01': 10, '012': 11, '01pm': 12, '02': 13, '020': 14, '021': 15, '023': 16, '02pm': 17, '03': 18, '032': 19, '033': 20, '037': 21, '04': 22, '04a': 23, '05': 24, '057': 25, '06': 26, '066': 27, '068': 28, '07': 29, '08': 30, '09': 31, '0964': 32, '0auda': 33, '0bama': 34, '0g': 35, '0j': 36, '0ncorpate': 37, '0o': 38, '0om': 39, '0onalo': 40, '10': 41, '100': 42, '1000': 43, '1000000': 44, '1001': 45, '100k': 46, '100m': 47, '101': 48, '102': 49, '1021': 50, '1022': 51, '102k': 52, '102spm': 53, '103': 54, '104': 55, '1046': 56, '1051': 57, '106': 58, '107': 59, '10am': 60, '10h': 61, '10k': 62, '10ntes': 63, '10th': 64, '11': 65, '110': 66, '1103': 67, '110k': 68, '111': 69, '11111': 70, '111111': 71, '112': 72, '1122k': 73, '113': 74, '115': 75, '116': 76, '117': 77, '118': 78, '119': 79, '119k': 80, '11alive': 81, '11h': 82, '11m': 83, '11s0': 84, '12': 85, '120': 86, '1200'